<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/DM_AOC_agent_orchestraTion_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Mar 23 02:12:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   43C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install --upgrade crewai langchain -q
!pip install --upgrade colab-env --quiet
!pip install --upgrade langchain_community -q
!pip install --upgrade faiss-cpu -q
!pip install --upgrade crewai-tools -q
!pip install 'crewai[tools]'
!pip install transformers -U -q

In [ ]:
import warnings

warnings.filterwarnings("ignore", message="You seem to be using the pipelines sequentially on GPU")

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from langchain_community.llms import HuggingFaceHub

In [ ]:
import colab_env
import os

In [ ]:
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [ ]:
from crewai import Agent, Task, Crew, Process
from litellm import completion

# Define Agents

# 1. Flight Status Monitor Agent
flight_monitor = Agent(
    role='Flight Status Monitor',
    goal='Continuously monitor real-time flight data and aircraft status for potential disruptions, including AOG situations.',
    backstory='You are a vigilant flight status monitor, always alert to potential disruptions and ready to trigger appropriate responses.',
    allow_delegation=False,
    verbose=True,
)

# 2. Crew Scheduler Agent
crew_scheduler = Agent(
    role='Crew Scheduler',
    goal='Adjust crew schedules in response to disruptions, ensuring flights are adequately staffed.',
    backstory='You are a skilled crew scheduler, adept at finding optimal solutions to crew-related disruptions, prioritizing both operational efficiency and crew well-being.',
    allow_delegation=False,
    verbose=True,
)

# 3. Passenger Communication Agent
passenger_communicator = Agent(
    role='Passenger Communication',
    goal='Communicate with passengers about disruptions, providing updates and alternative travel options.',
    backstory='You are a compassionate passenger communication specialist, dedicated to keeping passengers informed and supported during unexpected travel disruptions.',
    allow_delegation=False,
    verbose=True,
)

# 4. Resource Allocation Agent
resource_allocator = Agent(
    role='Resource Allocation',
    goal='Optimize the allocation of resources (e.g., aircraft, gates, ground staff) during disruptions, including AOG situations.',
    backstory='You are a strategic resource allocator, expertly managing resources to mitigate the impact of disruptions and maintain operational efficiency.',
    allow_delegation=False,
    verbose=True,
)

# 5. Maintenance Agent Scheduler Agent
maintenance_scheduler = Agent(
    role='Maintenance Agent Scheduler',
    goal='Schedule maintenance agents efficiently to address aircraft issues, prioritizing safety and minimizing operational disruptions, especially during AOG situations.',
    backstory="You are a skilled maintenance agent scheduler, adept at optimizing agent assignments based on their qualifications, aircraft needs, and operational constraints.",
    allow_delegation=False,
    verbose=True,
)

# 6. Dispatch - Flight Plan Agent
dispatch_planner = Agent(
    role='Dispatch - Flight Plan',
    goal='Generate and adjust flight plans, considering real-time conditions, aircraft performance, and regulatory requirements.',
    backstory="You are an experienced dispatch planner, responsible for creating safe and efficient flight plans, adapting to dynamic operational conditions.",
    allow_delegation=False,
    verbose=True,
)


# Define Tasks

# Flight Status Monitoring Tasks
monitor_flights_task = Task(
    description='Monitor real-time flight data for potential disruptions.',
    agent=flight_monitor,
    expected_output='Alerts about potential disruptions with relevant details.',
)

detect_aog_task = Task(
    description='Continuously monitor aircraft status and alert the system immediately when an AOG event is identified.',
    agent=flight_monitor,
    expected_output='AOG alert with relevant details (aircraft, location, reason).',
)

# Crew Scheduling Tasks
reschedule_crew_task = Task(
    description='Adjust crew schedules based on identified disruptions, ensuring adequate staffing for flights.',
    agent=crew_scheduler,
    expected_output='Updated crew schedules that minimize disruption impact.',
)

# Passenger Communication Tasks
notify_passengers_task = Task(
    description='Inform passengers about disruptions, provide updates, and offer alternative travel options.',
    agent=passenger_communicator,
    expected_output='Timely and informative passenger communications.',
)

notify_aog_passengers_task = Task(
    description='Inform passengers about the AOG situation, explain the expected impact on their travel plans, and offer alternative options such as rebooking or rerouting.',
    agent=passenger_communicator,
    expected_output='Clear and empathetic communication with passengers affected by the AOG.',
)

# Resource Allocation Tasks
optimize_resources_task = Task(
    description='Re-allocate resources (aircraft, gates, etc.) to minimize the overall impact of disruptions.',
    agent=resource_allocator,
    expected_output='Optimized resource allocation plans.',
)

optimize_aog_resources_task = Task(
    description='Re-allocate resources (aircraft, gates, crew) to minimize the cascading effects of the AOG on other flights and operations.',
    agent=resource_allocator,
    expected_output='Resource allocation plan adjusted for the AOG situation.',
)

# Maintenance Scheduling Tasks
prioritize_maintenance_task = Task(
    description='Prioritize incoming maintenance requests based on severity, aircraft type, and operational impact.',
    agent=maintenance_scheduler,
    expected_output='A prioritized list of maintenance requests.',
)

prioritize_aog_maintenance_task = Task(
    description='Prioritize maintenance requests related to the AOG situation based on urgency and criticality.',
    agent=maintenance_scheduler,
    expected_output='Prioritized list of AOG maintenance tasks.',
)


match_agent_skills_task = Task(
    description='Match maintenance requests to qualified agents based on their skills, certifications, and availability.',
    agent=maintenance_scheduler,
    expected_output='Assignments of maintenance agents to specific requests.',
)

optimize_maintenance_schedule_task = Task(
    description='Create an optimized schedule for maintenance agents, considering shift patterns, travel time, and workload balance.',
    agent=maintenance_scheduler,
    expected_output='A schedule that minimizes downtime and ensures efficient resource utilization.',
)

source_aog_parts_task = Task(
    description='Identify and source the necessary replacement parts for the AOG situation, coordinating with suppliers and managing logistics.',
    agent=maintenance_scheduler,
    expected_output='Confirmation of part sourcing and delivery arrangements.',
)

# Dispatch - Flight Plan Tasks
generate_flight_plan_task = Task(
    description='Generate initial flight plans based on route, aircraft type, weather forecasts, and air traffic control information.',
    agent=dispatch_planner,
    expected_output='A detailed flight plan with waypoints, altitudes, and fuel calculations.',
)

adjust_flight_plan_task = Task(
    description='Monitor real-time conditions (weather changes, airspace restrictions) and adjust flight plans as needed to ensure safety and efficiency.',
    agent=dispatch_planner,
    expected_output='Updated flight plans reflecting real-time conditions.',
)

optimize_fuel_task = Task(
    description='Analyze flight plans and optimize fuel consumption based on weather patterns, aircraft performance, and payload.',
    agent=dispatch_planner,
    expected_output='Flight plans with optimized fuel usage.',
)

communicate_with_crew_task = Task(
    description='Communicate flight plan updates and relevant information to the flight crew.',
    agent=dispatch_planner,
    expected_output='Clear and timely communication with the crew.',
)


# Initialize LLM
model_name = "huggingface/meta-llama/Llama-3.1-8B-Instruct"  # Use full Hugging Face path

# Define custom Agent.run method using litellm.completion
def agent_run_with_litellm(self, task_description):
    """
    This function replaces the default agent run behavior to use litellm.completion.
    """
    messages = [{"role": "user", "content": task_description}]
    response = completion(model=model_name, messages=messages)  # Use litellm.completion
    return response.choices[0].message.content  # Extract content from response

# Monkey patch the Agent.run method
Agent.run = agent_run_with_litellm


# Create and Run the Crew
crew = Crew(
    agents=[
        flight_monitor, crew_scheduler, passenger_communicator, resource_allocator,
        maintenance_scheduler, dispatch_planner
    ],
    tasks=[
        monitor_flights_task, detect_aog_task, reschedule_crew_task, notify_passengers_task, notify_aog_passengers_task,
        optimize_resources_task, optimize_aog_resources_task, prioritize_maintenance_task, prioritize_aog_maintenance_task,
        match_agent_skills_task, optimize_maintenance_schedule_task, source_aog_parts_task,
        generate_flight_plan_task, adjust_flight_plan_task, optimize_fuel_task, communicate_with_crew_task
    ],
    process=Process.sequential
)

try:
    result = crew.kickoff()
    print("Disruption Management Results:")
    print(result)

except Exception as e:
    print(f"An error occurred during disruption management: {e}")

# Agent: Flight Status Monitor
## Task: Monitor real-time flight data for potential disruptions.


# Agent: Flight Status Monitor
## Final Answer: 
As of the present monitoring period, the following potential disruptions have been identified in the current flight data:

1. **Flight Delay**: Flight AB123 from JFK to LAX has been delayed due to weather conditions. The estimated departure time has been pushed back by 45 minutes. Passengers are advised to check with the airline for updated information.

2. **Gate Change**: Flight CD456 from ORD to SFO has experienced a gate change from Gate 23 to Gate 31. Passengers should proceed to the new gate location to ensure timely boarding.

3. **AOG Situation**: Aircraft model XYZ 789 associated with Flight EF789 has been reported as AOG (Aircraft on Ground) due to a technical issue. The estimated time for the aircraft to return to service is currently undetermined. Affected passengers are being rebooked on alternative flights.

4. **Flight Cancel